In [ ]:
import numpy as np
import pandas as pd
import warnings
import shap
import matplotlib.pyplot as plt

In [ ]:
from geoshapley import GeoShapleyExplainer

In [ ]:
size=25
def plot_s(bs,vmin=None,vmax=None, title = ""):
    k = len(bs)
    fig, axs = plt.subplots(1, k,figsize=(6*k,4),dpi=300)
    for i in range(k):
        ax = axs[i].imshow(bs[i].reshape(size,size),cmap=plt.cm.get_cmap('viridis',36),
                               vmin=vmin,vmax=vmax)
        
        fig.colorbar(ax, ax=axs[i])

        axs[i].set_xticks(np.arange(-0.5, size, 5))
        axs[i].set_yticks(np.arange(-0.5, size, 5))
        axs[i].set_xticklabels([])
        axs[i].set_yticklabels([])
        axs[i].tick_params(axis='x', colors=(0,0,0,0))
        axs[i].tick_params(axis='y', colors=(0,0,0,0))
        
    fig.suptitle(title, fontsize=20,y=1.05)

In [ ]:
mgwr_sim = pd.read_csv("https://raw.githubusercontent.com/Ziqi-Li/geoshapley/refs/heads/main/data/mgwr_sim.csv")

In [ ]:
X_coords = mgwr_sim[['X1','X2','x_coord','y_coord']]
y = mgwr_sim.y.values

true = mgwr_sim[["b0","b1","b2"]].values

In [ ]:
warnings.filterwarnings('ignore')
plot_s(true.T,vmin=1,vmax=5,title = "True")

## Fit ML

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_coords, y,random_state=1)

In [ ]:
from hpsklearn import HyperoptEstimator, xgboost_regression, mlp_regressor
from hyperopt import tpe

In [ ]:
def train_any_model(any_regressor,max_evals=10):
    estim = HyperoptEstimator(regressor=any_regressor("myModel"),preprocessing=[],
                              algo=tpe.suggest,max_evals=max_evals,trial_timeout=60,seed=1)
    estim.fit(X_train, y_train)
    return estim

In [ ]:
%%time
for i in range(0,200):
    try:
        mlp_model = train_any_model(mlp_regressor)
        break
    except:
        pass

In [ ]:
%%time
xgb_model = train_any_model(xgboost_regression)

## Explain ML

In [ ]:
background_X = X_coords.values

### MLP

In [ ]:
mlp_explainer = GeoShapleyExplainer(mlp_model.predict, background_X)

mlp_rslt = mlp_explainer.explain(X_coords,n_jobs=-1)

In [ ]:
mlp_rslt.summary_plot(dpi=150)

In [ ]:
mlp_rslt.summary_statistics()

### XGBoost

In [ ]:
xgb_explainer = GeoShapleyExplainer(xgb_model.predict, background_X)

xgb_rslt = xgb_explainer.explain(X_coords, n_jobs=-1)

In [ ]:
xgb_rslt.summary_statistics

In [ ]:
mlp_rslt.summary_plot(dpi=150)

In [ ]:
xgb_svc = xgb_rslt.get_svc(col = [0, 1],coef_type="raw",include_primary=True)

plot_s(np.hstack([xgb_rslt.base_value + xgb_rslt.geo.reshape(-1,1), xgb_svc]).T,
       vmin=1,vmax=5,title="XGBoost Explanations")

In [ ]:
mlp_svc = mlp_rslt.get_svc(col = [0, 1],coef_type="raw",include_primary=True)

In [ ]:
plot_s(np.hstack([mlp_rslt.base_value + mlp_rslt.geo.reshape(-1,1), mlp_svc]).T,
       vmin=1,vmax=5,title="MLP Explanations")

In [ ]:
mlp_svc = mlp_rslt.get_svc(col = [0, 1],coef_type="gwr",include_primary=True)

plot_s(np.hstack([mlp_rslt.base_value + mlp_rslt.geo.reshape(-1,1), mlp_svc]).T,
       vmin=1,vmax=5,title="MLP Explanations - Smoothed")

## MGWR

In [ ]:
from mgwr.gwr import GWR,MGWR
from mgwr.sel_bw import Sel_BW

In [ ]:
sel = Sel_BW(X_coords.values[:,-2:],y.reshape(-1,1),X_coords.values[:,:-2],multi=True)
sel.search()

mgwr_rslt = MGWR(X_coords.values[:,-2:],y.reshape(-1,1),X_coords.values[:,:-2],selector=sel).fit()

In [ ]:
plot_s(mgwr_rslt.params.T,vmin=1,vmax=5,title="MGWR estimates")